In [1]:
import pandas as pd
import re
import pickle
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM, Dropout,Bidirectional
from keras.layers.embeddings import Embedding
from keras.preprocessing import sequence
from keras.models import load_model
import numpy as np

Using TensorFlow backend.


In [0]:
data= pd.read_csv('IMDB Dataset.csv')

In [3]:
data

,review,sentiment
0,One of the other reviewers has mentioned that ...,positive
1,A wonderful little production. <br /><br />The...,positive
2,I thought this was a wonderful way to spend ti...,positive
3,Basically there's a family where a little boy ...,negative
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive
...,...,...
49995,I thought this movie did a down right good job...,positive
49996,"Bad plot, bad dialogue, bad acting, idiotic di...",negative
49997,I am a Catholic taught in parochial elementary...,negative
49998,I'm going to have to disagree with the previou...,negative


In [0]:
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
stemmer=WordNetLemmatizer()
corpus=[]
for i in range(0,len(data.review)):
    review = re.sub('[^a-zA-Z]', ' ',data['review'][i])
    review = review.lower()
    review = review.split()
    
    review = [stemmer.lemmatize(word) for word in review if not word in stopwords.words('english')]
    review = ' '.join(review)
    corpus.append(review)

In [0]:
from keras.preprocessing.text import one_hot
voc_size=5000
one_hot_rep=[one_hot(i,voc_size) for i in corpus]

In [0]:
X=one_hot_rep
y=data.sentiment
from sklearn.preprocessing import LabelEncoder
encoder=LabelEncoder()
y=encoder.fit_transform(y)

In [0]:
from sklearn.model_selection import train_test_split
x_train,x_tot,y_train,y_tot=train_test_split(X,y,test_size=0.2,random_state=101)
x_val,x_test,y_val,y_test=train_test_split(x_tot,y_tot,test_size=0.5,random_state=101)

In [0]:
max_review_length = 500
x_train = sequence.pad_sequences(x_train, maxlen=max_review_length)
x_val= sequence.pad_sequences(x_val, maxlen=max_review_length)
x_test= sequence.pad_sequences(x_test, maxlen=max_review_length)

In [10]:
x_val.shape

(5000, 500)

In [11]:
embedding_vecor_length = 32
model = Sequential()
model.add(Embedding(voc_size, embedding_vecor_length, input_length=max_review_length))
model.add(Dropout(0.2))
model.add(Bidirectional(LSTM(100)))
model.add(Dropout(0.2))
model.add(Dense(1, activation='sigmoid'))

model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 500, 32)           160000    
_________________________________________________________________
dropout_1 (Dropout)          (None, 500, 32)           0         
_________________________________________________________________
bidirectional_1 (Bidirection (None, 200)               106400    
_________________________________________________________________
dropout_2 (Dropout)          (None, 200)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 201       
Total params: 266,601
Trainable params: 266,601
Non-trainable params: 0
_________________________________________________________________


In [12]:
model.fit(x=x_train,y=y_train,validation_data=(x_val,y_val),batch_size=512,epochs=10)

/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/indexed_slices.py:434: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Train on 40000 samples, validate on 5000 samples
Epoch 1/10
40000/40000 [==============================] - 192s 5ms/step - loss: 0.6504 - accuracy: 0.6486 - val_loss: 0.5055 - val_accuracy: 0.7970
Epoch 2/10
40000/40000 [==============================] - 192s 5ms/step - loss: 0.3768 - accuracy: 0.8368 - val_loss: 0.3409 - val_accuracy: 0.8534
Epoch 3/10
40000/40000 [==============================] - 190s 5ms/step - loss: 0.3032 - accuracy: 0.8756 - val_loss: 0.3184 - val_accuracy: 0.8690
Epoch 4/10
40000/40000 [==============================] - 188s 5ms/step - loss: 0.2745 - accuracy: 0.8906 - val_loss: 0.3229 - val_accuracy: 0.8640
Epoch 5/10
40000/40000 [==============================] - 190s 5ms/step - loss: 0.2597 - accuracy: 0.8974 - val_loss: 0.3259 - val_accuracy: 0.8638
Epoch 6/10
40000/40000 [==============================] - 188s 5ms/step - loss: 0.2528 - accuracy: 0.9001 - val_loss: 0.3317 - val_accuracy: 0.8668
Epoch 7/10
40000/40000 [==============================] - 191s 

In [13]:
model.evaluate(x_test,y_test)

5000/5000 [==============================] - 24s 5ms/step


[0.3479141173005104, 0.8628000020980835]

In [0]:
model.save('sentiment_model.h5')